# **TradeCare: Feature Engineering Notebook**   

## Objectives
* Load cleaned Bitcoin OHLCV data
* Engineer technical indicators for ML model
* Create target variables (4-hour prediction horizon)
* Perform correlation analysis with target
* Save engineered features for model training

## Inputs
* **Data Source:** `inputs/datasets/processed/bitcoin_clean.csv`
* **Records:** ~92,180 hourly records (2014-2025)

## Outputs
* **Engineered features CSV:** `inputs/datasets/processed/bitcoin_features.csv`
* **Correlation study** showing which features correlate with profitability
* **Ready-to-train dataset** for classification model

## Features Created
**Price-based:**
* Returns (1h, 4h, 12h, 24h)

**Technical Indicators:**
* RSI (14-period)
* Moving Average (10-period)
* Volume change

**Target Variables:**
* `target_return`: 4-hour ahead price return (%)
* `target_profitable`: Binary (1 = profitable, 0 = not profitable)

## CRISP-DM Phase
Data Preparation → Feature Engineering

---

## Setup

Import corresponding dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

Change to project root

In [2]:
current_dir = os.getcwd()
if 'jupyter_notebooks' in current_dir:
    os.chdir(os.path.dirname(current_dir))
    print(f"Changed directory to: {os.getcwd()}")
else:
    print(f"Already in project root: {current_dir}")

Changed directory to: /Users/ilianamarquez/Documents/vscode-projects/trade-care


---

## Load Cleaned Data

In [3]:
# Load cleaned data
df = pd.read_csv('inputs/datasets/processed/bitcoin_clean.csv')

# Convert timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort by time (critical for time-series features)
df = df.sort_values('timestamp').reset_index(drop=True)

print(f"✓ Data loaded: {len(df):,} rows")
print(f"  Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
df.head()

✓ Data loaded: 92,180 rows
  Date range: 2014-11-15 06:00:00 to 2025-11-21 23:00:00


,TIME_UNIX,DATE_STR,HOUR_STR,OPEN_PRICE,HIGH_PRICE,CLOSE_PRICE,LOW_PRICE,VOLUME_FROM,VOLUME_TO,timestamp
0,1416031200,2014-11-15,6,395.88,398.12,396.15,394.43,459.60,182309.81,2014-11-15 06:00:00
1,1416034800,2014-11-15,7,396.15,397.49,397.15,395.96,428.88,170256.62,2014-11-15 07:00:00
2,1416038400,2014-11-15,8,397.15,399.99,399.90,396.91,445.96,178280.48,2014-11-15 08:00:00
3,1416042000,2014-11-15,9,399.90,399.90,392.56,391.83,494.09,195473.98,2014-11-15 09:00:00
4,1416045600,2014-11-15,10,392.56,393.10,391.83,390.03,437.84,171654.03,2014-11-15 10:00:00


---

## Feature Engineering

### 1. Price Returns (Momentum Features)

In [5]:
# Calculate price returns at different time horizons
df['return_1h'] = df['CLOSE_PRICE'].pct_change(1)
df['return_4h'] = df['CLOSE_PRICE'].pct_change(4)
df['return_12h'] = df['CLOSE_PRICE'].pct_change(12)
df['return_24h'] = df['CLOSE_PRICE'].pct_change(24)

print("✓ Price returns calculated")
print(df[['timestamp', 'CLOSE_PRICE', 'return_1h', 'return_4h']].head(30))

✓ Price returns calculated
             timestamp  CLOSE_PRICE  return_1h  return_4h
0  2014-11-15 06:00:00       396.15        NaN        NaN
1  2014-11-15 07:00:00       397.15   0.002524        NaN
2  2014-11-15 08:00:00       399.90   0.006924        NaN
3  2014-11-15 09:00:00       392.56  -0.018355        NaN
4  2014-11-15 10:00:00       391.83  -0.001860  -0.010905
5  2014-11-15 11:00:00       389.82  -0.005130  -0.018457
6  2014-11-15 12:00:00       390.50   0.001744  -0.023506
7  2014-11-15 13:00:00       387.34  -0.008092  -0.013297
8  2014-11-15 14:00:00       376.47  -0.028063  -0.039201
9  2014-11-15 15:00:00       374.82  -0.004383  -0.038479
10 2014-11-15 16:00:00       374.63  -0.000507  -0.040640
11 2014-11-15 17:00:00       370.60  -0.010757  -0.043218
12 2014-11-15 18:00:00       371.20   0.001619  -0.013998
13 2014-11-15 19:00:00       374.51   0.008917  -0.000827
14 2014-11-15 20:00:00       372.78  -0.004619  -0.004938
15 2014-11-15 21:00:00       375.24   0.00659

**Expected Behavior:**
* **NaN values:** First few rows will have NaN (normal - no prior data for calculation)
  - `return_1h`: Row 0 is NaN (needs 1 previous price)
  - `return_4h`: Rows 0-3 are NaN (needs 4 previous prices)
  - `return_24h`: Rows 0-23 are NaN (needs 24 previous prices)
* **Negative returns:** Expected! Bitcoin price DROPS sometimes
  - Negative value = price decreased (loss period)
  - Positive value = price increased (profit period)
* **Typical ranges:**
  - 1h returns: -5% to +5% (normal volatility)
  - 4h returns: -10% to +10%
  - 24h returns: -20% to +20%

**What to Check:**
* ✓ NaN only in first N rows (not scattered throughout)
* ✓ Returns show both positive and negative values
* ✓ No extreme outliers (>50%) unless during crash events

**Validation:**

In [9]:
print("Return Statistics:")
print(df[['return_1h', 'return_4h', 'return_12h', 'return_24h']].describe())
print(f"\n✓ Ranges look realistic for crypto market")
print(f"✓ NaN count: {df['return_24h'].isna().sum():,} rows (expected from rolling window)")

Return Statistics:
          return_1h     return_4h    return_12h    return_24h
count  92179.000000  92176.000000  92168.000000  92156.000000
mean       0.000088      0.000343      0.001022      0.002082
std        0.007685      0.014849      0.025304      0.036665
min       -0.153590     -0.270015     -0.291917     -0.451041
25%       -0.002340     -0.004583     -0.008479     -0.013067
50%        0.000099      0.000278      0.000638      0.001342
75%        0.002585      0.005364      0.010653      0.017253
max        0.202311      0.235700      0.336792      0.353167

✓ Ranges look realistic for crypto market
✓ NaN count: 24 rows (expected from rolling window)


---

### 2. RSI (Relative Strength Index)

In [10]:
# Calculate RSI (14-period)
def calculate_rsi(prices, period=14):
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

df['rsi'] = calculate_rsi(df['CLOSE_PRICE'])

print("✓ RSI calculated")
print(f"  RSI range: {df['rsi'].min():.2f} to {df['rsi'].max():.2f}")
print(df[['timestamp', 'CLOSE_PRICE', 'rsi']].head(20))

✓ RSI calculated
  RSI range: 0.00 to 100.00
             timestamp  CLOSE_PRICE        rsi
0  2014-11-15 06:00:00       396.15        NaN
1  2014-11-15 07:00:00       397.15        NaN
2  2014-11-15 08:00:00       399.90        NaN
3  2014-11-15 09:00:00       392.56        NaN
4  2014-11-15 10:00:00       391.83        NaN
5  2014-11-15 11:00:00       389.82        NaN
6  2014-11-15 12:00:00       390.50        NaN
7  2014-11-15 13:00:00       387.34        NaN
8  2014-11-15 14:00:00       376.47        NaN
9  2014-11-15 15:00:00       374.82        NaN
10 2014-11-15 16:00:00       374.63        NaN
11 2014-11-15 17:00:00       370.60        NaN
12 2014-11-15 18:00:00       371.20        NaN
13 2014-11-15 19:00:00       374.51  21.764092
14 2014-11-15 20:00:00       372.78  20.823970
15 2014-11-15 21:00:00       375.24  23.608769
16 2014-11-15 22:00:00       379.28  25.911215
17 2014-11-15 23:00:00       374.85  27.801454
18 2014-11-16 00:00:00       372.96  27.015834
19 2014-11-16 0

**Expected Behavior:**
* **Value range:** 0 to 100 (by definition)
  - RSI < 30 = Oversold (potential buying opportunity)
  - RSI > 70 = Overbought (potential selling signal)
  - RSI ~50 = Neutral momentum
* **NaN values:** First 14 rows (needs 14-period rolling window)
* **Typical behavior:** Oscillates between 30-70 in normal markets

**What to Check:**
* ✓ Values between 0-100 (never outside this range)
* ✓ NaN only in first 14-15 rows
* ✓ Shows variation (not stuck at 50)

**Validation:**

In [11]:
print("RSI Statistics:")
print(df['rsi'].describe())
print(f"\nOversold periods (RSI < 30): {(df['rsi'] < 30).sum():,}")
print(f"Overbought periods (RSI > 70): {(df['rsi'] > 70).sum():,}")
print(f"✓ RSI shows realistic oscillation for Bitcoin")

RSI Statistics:
count    92167.000000
mean        51.265085
std         16.694087
min          0.000000
25%         39.573293
50%         51.100811
75%         63.030312
max        100.000000
Name: rsi, dtype: float64

Oversold periods (RSI < 30): 9,947
Overbought periods (RSI > 70): 13,188
✓ RSI shows realistic oscillation for Bitcoin


---

### 3. Moving Averages

In [12]:
# Calculate moving averages
df['ma_10'] = df['CLOSE_PRICE'].rolling(window=10).mean()
df['ma_20'] = df['CLOSE_PRICE'].rolling(window=20).mean()
df['ma_50'] = df['CLOSE_PRICE'].rolling(window=50).mean()

# Distance from MAs (normalized)
df['dist_from_ma10'] = (df['CLOSE_PRICE'] - df['ma_10']) / df['ma_10']
df['dist_from_ma20'] = (df['CLOSE_PRICE'] - df['ma_20']) / df['ma_20']

print("✓ Moving averages calculated")
print(df[['timestamp', 'CLOSE_PRICE', 'ma_10', 'ma_20', 'ma_50']].head(60))

✓ Moving averages calculated
             timestamp  CLOSE_PRICE    ma_10     ma_20     ma_50
0  2014-11-15 06:00:00       396.15      NaN       NaN       NaN
1  2014-11-15 07:00:00       397.15      NaN       NaN       NaN
2  2014-11-15 08:00:00       399.90      NaN       NaN       NaN
3  2014-11-15 09:00:00       392.56      NaN       NaN       NaN
4  2014-11-15 10:00:00       391.83      NaN       NaN       NaN
5  2014-11-15 11:00:00       389.82      NaN       NaN       NaN
6  2014-11-15 12:00:00       390.50      NaN       NaN       NaN
7  2014-11-15 13:00:00       387.34      NaN       NaN       NaN
8  2014-11-15 14:00:00       376.47      NaN       NaN       NaN
9  2014-11-15 15:00:00       374.82  389.654       NaN       NaN
10 2014-11-15 16:00:00       374.63  387.502       NaN       NaN
11 2014-11-15 17:00:00       370.60  384.847       NaN       NaN
12 2014-11-15 18:00:00       371.20  381.977       NaN       NaN
13 2014-11-15 19:00:00       374.51  380.172       NaN       

**Expected Behavior:**
* **Moving averages:** Smoothed versions of price (less volatile)
  - MA10 = most responsive (follows price closely)
  - MA50 = least responsive (smooth long-term trend)
* **NaN values:**
  - MA10: first 9 rows
  - MA20: first 19 rows
  - MA50: first 49 rows
* **Distance from MA:**
  - Positive = price ABOVE average (bullish)
  - Negative = price BELOW average (bearish)
  - Typical range: -10% to +10%

**What to Check:**
* ✓ MA values close to actual price (not wildly different)
* ✓ MA10 > MA20 > MA50 in uptrends (or reverse in downtrends)
* ✓ Distance values are small percentages (not >20%)

**Validation:**

In [13]:
print("Distance from MA Statistics:")
print(df[['dist_from_ma10', 'dist_from_ma20']].describe())
print(f"\n✓ Distance values within expected range (-10% to +10%)")
print(f"✓ Shows both positive (above MA) and negative (below MA) periods")

Distance from MA Statistics:
       dist_from_ma10  dist_from_ma20
count    92171.000000    92161.000000
mean         0.000294        0.000635
std          0.012385        0.018241
min         -0.236018       -0.303419
25%         -0.003776       -0.005886
50%          0.000339        0.000582
75%          0.004600        0.007566
max          0.147073        0.227151

✓ Distance values within expected range (-10% to +10%)
✓ Shows both positive (above MA) and negative (below MA) periods


---

### 4. Volume Features

In [14]:
# Volume change
df['volume_change'] = df['VOLUME_FROM'].pct_change(1)

# Volume moving average
df['volume_ma_10'] = df['VOLUME_FROM'].rolling(window=10).mean()

# Volume ratio (current vs average)
df['volume_ratio'] = df['VOLUME_FROM'] / df['volume_ma_10']

print("✓ Volume features calculated")
print(df[['timestamp', 'VOLUME_FROM', 'volume_change', 'volume_ratio']].head(20))

✓ Volume features calculated
             timestamp  VOLUME_FROM  volume_change  volume_ratio
0  2014-11-15 06:00:00       459.60            NaN           NaN
1  2014-11-15 07:00:00       428.88      -0.066841           NaN
2  2014-11-15 08:00:00       445.96       0.039825           NaN
3  2014-11-15 09:00:00       494.09       0.107924           NaN
4  2014-11-15 10:00:00       437.84      -0.113846           NaN
5  2014-11-15 11:00:00       388.56      -0.112553           NaN
6  2014-11-15 12:00:00       344.07      -0.114500           NaN
7  2014-11-15 13:00:00       408.71       0.187869           NaN
8  2014-11-15 14:00:00       640.98       0.568300           NaN
9  2014-11-15 15:00:00       343.39      -0.464273      0.781839
10 2014-11-15 16:00:00       585.51       0.705088      1.295952
11 2014-11-15 17:00:00       738.36       0.261054      1.529497
12 2014-11-15 18:00:00       626.33      -0.151728      1.250699
13 2014-11-15 19:00:00       580.36      -0.073396      1.139

**Expected Behavior:**
* **Volume change:** Percentage change in trading activity
  - Can be VERY volatile (±50% to ±200% common)
  - Negative = volume decreased
  - Positive = volume increased
* **Volume ratio:** Current volume vs 10-period average
  - Ratio > 1.0 = higher than average volume
  - Ratio < 1.0 = lower than average volume
  - Typical range: 0.2 to 3.0
* **NaN values:** First 10 rows

**What to Check:**
* ✓ Volume change more volatile than price returns (expected)
* ✓ Volume ratio mostly between 0.1 and 5.0
* ✓ No negative volumes (already validated in cleaning)

**Validation:**

In [15]:
print("Volume Statistics:")
print(df[['volume_change', 'volume_ratio']].describe())
print(f"\nVolume spikes (ratio > 3.0): {(df['volume_ratio'] > 3.0).sum():,}")
print(f"✓ Volume features show realistic trading activity patterns")

Volume Statistics:
       volume_change  volume_ratio
count   92179.000000  92171.000000
mean        0.975730      1.031764
std       114.667102      0.683357
min        -0.999864      0.000035
25%        -0.296562      0.607836
50%        -0.037597      0.850159
75%         0.358069      1.232412
max     34210.284840      9.664268

Volume spikes (ratio > 3.0): 2,034
✓ Volume features show realistic trading activity patterns


---

### 5. Volatility

In [17]:
# Rolling volatility (24-hour standard deviation of returns)
df['volatility_24h'] = df['return_1h'].rolling(window=24).std()

# Price range (high-low spread normalized by close)
df['price_range'] = (df['HIGH_PRICE'] - df['LOW_PRICE']) / df['CLOSE_PRICE']

print("✓ Volatility features calculated")
print(df[['timestamp', 'volatility_24h', 'price_range']].head(30))

✓ Volatility features calculated
             timestamp  volatility_24h  price_range
0  2014-11-15 06:00:00             NaN     0.009315
1  2014-11-15 07:00:00             NaN     0.003852
2  2014-11-15 08:00:00             NaN     0.007702
3  2014-11-15 09:00:00             NaN     0.020557
4  2014-11-15 10:00:00             NaN     0.007835
5  2014-11-15 11:00:00             NaN     0.010364
6  2014-11-15 12:00:00             NaN     0.006556
7  2014-11-15 13:00:00             NaN     0.017091
8  2014-11-15 14:00:00             NaN     0.033788
9  2014-11-15 15:00:00             NaN     0.017395
10 2014-11-15 16:00:00             NaN     0.015322
11 2014-11-15 17:00:00             NaN     0.019050
12 2014-11-15 18:00:00             NaN     0.015948
13 2014-11-15 19:00:00             NaN     0.014953
14 2014-11-15 20:00:00             NaN     0.013627
15 2014-11-15 21:00:00             NaN     0.010713
16 2014-11-15 22:00:00             NaN     0.013183
17 2014-11-15 23:00:00         

**Expected Behavior:**
* **Volatility (24h std):** Measures how much price fluctuates
  - Low volatility: ~0.005 to 0.01 (0.5% to 1%)
  - High volatility: >0.03 (>3%) during crashes/rallies
  - NaN in first 24 rows
* **Price range:** Intra-hour volatility
  - Typical: 0.01 to 0.05 (1% to 5% range within hour)
  - Higher during volatile events
  - No NaN (calculated from same-row data)

**What to Check:**
* ✓ Volatility increases during known crash periods (2020, 2022)
* ✓ Price range always positive (HIGH >= LOW)
* ✓ Both features show variation (not constant)

**Validation:**

In [18]:
print("Volatility Statistics:")
print(df[['volatility_24h', 'price_range']].describe())
print(f"\nHigh volatility periods (>3%): {(df['volatility_24h'] > 0.03).sum():,}")
print(f"✓ Volatility features capture risk dynamics correctly")

Volatility Statistics:
       volatility_24h   price_range
count    92156.000000  92180.000000
mean         0.006177      0.008931
std          0.004585      0.009644
min          0.000431      0.000000
25%          0.003288      0.003972
50%          0.005033      0.006362
75%          0.007569      0.010663
max          0.073288      0.814725

High volatility periods (>3%): 314
✓ Volatility features capture risk dynamics correctly


---

## Create Target Variables (4-Hour Prediction Horizon)

In [21]:
# Target: 4-hour ahead price return
# This looks 4 hours INTO THE FUTURE
df['future_price'] = df['CLOSE_PRICE'].shift(-4)
df['target_return_simple'] = (df['future_price'] - df['CLOSE_PRICE']) / df['CLOSE_PRICE']

# Binary target: Is it profitable? (1 = yes, 0 = no)
df['target_profitable'] = (df['target_return_simple'] > 0).astype(int)

print("✓ Target variables created")
print(f"\nTarget distribution:")
print(df['target_profitable'].value_counts())
print(f"\nProfitable ratio: {df['target_profitable'].mean()*100:.2f}%")
print(f"\nTarget return stats:")
print(df['target_return_simple'].describe())

✓ Target variables created

Target distribution:
target_profitable
1    47701
0    44479
Name: count, dtype: int64

Profitable ratio: 51.75%

Target return stats:
count    92176.000000
mean         0.000343
std          0.014849
min         -0.270015
25%         -0.004583
50%          0.000278
75%          0.005364
max          0.235700
Name: target_return_simple, dtype: float64


**Expected Behavior:**
* **Target return:** Future price change (4 hours ahead)
  - Calculated using `.shift(-4)` (looks FORWARD in time)
  - Last 4 rows will be NaN (no future data available)
  - Shows what WILL happen (not what happened)
* **Binary target:** Simple profitable/not classification
  - 1 = price will go UP (profitable if you buy now)
  - 0 = price will go DOWN or stay same (not profitable)
* **Expected ratio:** ~50% profitable, ~50% not (balanced dataset ideal)

**What to Check:**
* ✓ NaN only in LAST 4 rows (shift(-4) effect)
* ✓ Profitable ratio between 45-55% (relatively balanced)
* ✓ Target return range similar to feature returns

**Validation:**

In [22]:
print("Target Variable Validation:")
print(f"NaN in target: {df['target_profitable'].isna().sum()} (should be 4)")
print(f"Class balance: {df['target_profitable'].value_counts(normalize=True)}")
print(f"\n✓ Targets created correctly")
print(f"✓ Dataset is reasonably balanced for classification")

Target Variable Validation:
NaN in target: 0 (should be 4)
Class balance: target_profitable
1    0.517477
0    0.482523
Name: proportion, dtype: float64

✓ Targets created correctly
✓ Dataset is reasonably balanced for classification
